In [4]:
import os
import json
import tracemalloc
import pandas as pd

In [19]:
from dotenv import load_dotenv
load_dotenv()

True

In [20]:
key=os.getenv("OPENAI_API_KEY")

In [23]:
if key:
    print("Key loaded successfully")
else:
    print("Key not found")

Key loaded successfully


In [22]:
from langchain.chat_models import ChatOpenAI
llm=ChatOpenAI(openai_api_key=key,model_name='gpt-3.5-turbo', temperature=0.5)